# Übung zu Kapitel 4.1 zur Datenvorbereitung

*Eine Übung zum Buch "[Basiswissen KI-Testen - Qualität von und mit KI-basierten Systemen](https://dpunkt.de/produkt/basiswissen-ki-testen/)", ISBN 978-3-86490-947-4*

In dieser Übung verwenden wir den *Iris*-Datensatz, der häufig zur Einführung für ML-Grundlagen verwendet wird. Wenn wir den Datensatz beispielsweise bei [openML](https://www.openml.org/search?type=data&status=active&id=61&sort=runs) herunterladen, ist der Datensatz schon soweit vorbereitet, dass man ihn ohne weitere Aufbereitung zum ML verwenden kann. In der Realität sehen Rohdaten selten so aus.

Für diese Übung haben wir einen Datensatz vorbereitet, der einige Fallstricke enthält, die häufig in Rohdaten zu finden sind. In diesem Notebook bereiten wir den Rohdatensatz soweit auf, dass wir ihn in den folgenden Übungen zur Erstellung eines ML-Modells verwenden können.
Wir nutzen hier wieder mehrere Bibliotheken, die dafür passende Methoden bereithalten:

[<img src="https://pandas.pydata.org/docs/_static/pandas.svg" alt="pandas" width="80" height="24">](https://pandas.pydata.org/docs/reference/index.html)
&emsp; [<img src="https://numpy.org/doc/stable/_static/numpylogo.svg" alt="Numpy" width="80" height="24">](https://numpy.org/doc/stable/reference/index.html#reference)
&emsp; [<img src="https://docs.scipy.org/doc/scipy/_static/logo.svg" alt="SciPy" width="24" height="24"> SciPy](https://docs.scipy.org/doc/scipy/index.html)

In dieser Übung lesen wir einen Datensatz ein und schauen uns die folgenden Aspekte in der Vorverarbeitung der Daten genauer an:
- fehlerhafte Einträge
- Ausreißer
- Abschätzung fehlender Einträge
- Duplikate
- überflüssige Merkmale
- kategorischer Daten in numerische Daten

## Aufgabe 1
**Lies den Iris-Datensatz aus der CSV-Datei ein und analysiere, ob die eingelesenen Informationen konsistent sind. Korrigiere diese gegebenenfalls.**

#### Datensatz einlesen

In [ ]:
# importieren der Bibliotheken, die wir für die Aufgaben 1 und 2 benötigen
import pandas as pd
import numpy  as np

Wir haben dir eine csv-Datei (csv steht für Comma-Separated-Values-Format) vorbereitet und lesen dieses File mit der [*read_csv*](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)-Funktion aus der Pandas-Bibliothek ein.

In [ ]:
data = pd.read_csv("Iris.csv")

Um sicherzugehen, ob das Einlesen der Datei richtig funktioniert hat, schauen wir uns mit der [*head*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)-Funktion die ersten Zeilen des Iris-Datensatzes an.

In [ ]:
data.head()

Oh, hier ist etwas schiefgelaufen: Beim Einlesen wurden die Spalten nicht richtig erkannt. In der ersten Zeile sehen wir, dass die Spalten "counter", "sepallength", "sepalwidth", "petallength", "petalwidth" und "class" nicht durch Kommata sondern durch Semikola (;) voneinander getrennt werden. 

**Aufgabe:**
Gib in der [*read_csv*](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)-Funktion an, das in unserer Datei das Semikolon als Separator verwendet wurden soll.

In [ ]:
data = pd.read_csv("Iris.csv", sep = ...)
data.head()

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung01.py

Wenn wir uns die Einträge in den Daten anschauen, fällt gleich auf, dass bei der Datenerhebung als Dezimaltrennzeichen Kommata benutzt wurden. Python verwendet aber Punkte als Dezimaltrennzeichen. Damit wir mit den Daten in Python weiterarbeiten können, müssen wir einheitlich Punkte verwenden.

**Aufgabe:**
Gib in der [*read_csv*](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) an, dass in unserer Datei das Komma als Dezimaltrennzeichen (englisch: decimal separator) verwendet werden soll.

In [ ]:
data = pd.read_csv("Iris.csv", sep = ";", decimal=...)
data.head()

In [ ]:
# Um die Lösung anzuzeigen, bitte diese Zelle zweimal ausführen
%load Lösungen/Lösung02.py

#### Erste Datenanalyse

Mit der [*describe*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)-Funktion können wir mit einem Blick einen guten Überblick über die Daten erhalten. So können wir schnell erste Hinweise auf notwendige Schritte in der Datenvorbereitung finden.

In [ ]:
data.describe()

Hier fallen uns ein paar Dinge auf:
- Der Maximalwert für petallength ist mit 13 auffallend hoch und weit entfernt von den anderen Werten. -> Ist das realistisch?
- Die Anzahl der Einträge, angegeben in der Zeile "count", ist für die verschiedenen Spalten unterschiedlich. -> Fehlen hier Einträge?

## Aufgabe 2
**Finde fehlerhafte Einträge mit numerischen Ausreißern oder fehlenden Werten und entferne oder korrigieren diese**

#### Extremwerte

Als erstes schauen wir uns die Datenreihe mit *sepalwidth* von 13 cm genauer an. 

In [ ]:
data[data["sepalwidth"] == 13]

In [ ]:
data["sepalwidth"].sort_values(ascending= False)

Die Datenreihe mit der sepalwidth von 13 ist ein Ausreißer nach oben (er hat gleichzeitig auch den niedrigsten Wert für das Merkmal petallength). Diese Datenreihe verwerfen wir besser. Dazu wenden wir die Methode [*drop*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html#pandas.DataFrame.drop) auf den Eintrag mit der Nummer 100 an (Zähler der originalen Einträge in der ersten Spalte).

In [ ]:
data = data.drop(labels = 100, axis = 0)

#### Fehlende/Fehlerhafte Daten

Nun schauen wir nach *fehlenden* Einträgen. Dazu verwenden wir die Funktion [*isna*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html).

In [ ]:
data[data.isna().any(axis=1)]

Der Eintrag mit dem Index 138 ist eine leere Datenreihe. Diese Zeile entfernen wir aus dem Datensatz.

In [ ]:
data = data.drop(labels = 138, axis = 0)
data[data.isna().any(axis=1)]

Bei den anderen fehlenden Einträgen wenden wir eine andere Methode an: Wir ersetzen diese durch Mittelwerte.
Die fehlenden Werte für den 25sten und 106ten Eintrag können wir gut durch den jeweiligen Mittelwert der entsprechenden Schwertlienienart (auf eine Nachkommastelle) abschätzen.

In [ ]:
data[data["class"] == "Iris-setosa"]["sepallength"].mean()

In [ ]:
data.loc[25,'sepallength'] = 5

In [ ]:
data[data["class"] == "Iris-virginica"]["petallength"].mean()

In [ ]:
data.loc[106,'petallength'] = 5.6

Wir können abschließend noch überprüfen, ob die beiden letzten Änderungen erfolgreich waren und es keine fehlenden Einträge (NaN) mehr gibt.

In [ ]:
data.loc[[25,106]]

In [ ]:
data[data.isna().any(axis=1)]

## Aufgabe 3
**Finde und entferne irrelevante Informationen wie Duplikate oder überflüssige Spalten**

#### Duplikate entfernen

Duplikate in Datensätzen können im Training einen Bias verursachen. Daher wollen wir Duplikate finden und entfernen.

In [ ]:
data[data.duplicated()]

In [ ]:
data.loc[84:89]

Hier wurden die Dateneinträge mit dem **counter** 86 und 87 zweimal zu dem Datensatz hinzugefügt. Wir entfernen die Datenreihen mit der [drop_duplicates](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)-Funktion.

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.loc[84:89]

#### Überflüssige Spalte löschen

Informationen in den Daten, die nicht mit dem Datensatz direkt in Verbindung stehen sondern mit der Datenerhebung wie hier die Spalte "counter" entfernen wir. So verhindern wir, dass das ML-Modell später diese Informationen lernt und so das Lernen negativ beeinträchtigt wird.

In [ ]:
data = data.drop(columns = ['counter'])
data

## Aufgabe 4

#### Wandle kategorische Merkmale in numerische um

ML-Modelle arbeiten mit numerischen und Daten, daher wandeln wir die kategorischen Klassen 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica' in einen numerischen Code um. Die Funktion [*unique*](https://pandas.pydata.org/docs/reference/api/pandas.unique.html) gibt dabei von jedem Wert (hier die Namen der Iris-Klassen) nur einen zurück.

In [ ]:
np.unique(data['class'])

In [ ]:
data = data.replace('Iris-setosa', 0)
data = data.replace('Iris-versicolor',1)
data = data.replace('Iris-virginica',2)

## Test und Speichern
**Test, ob alle Fallstricke gefunden wurden**

Wir prüfen nun, ob alle deine Schritte richtig waren, indem wir den aufbereiteten Datensatz `data` mit einer Musterlösung vergleichen.

In [ ]:
from scipy.io import arff
pdata, meta = arff.loadarff('Lösungen/dataset_61_iris_lösung.arff')   # die Daten in 'pdata' einlesen; 'meta' enthält Metainformationen 
df = pd.DataFrame(pdata)                                              # die Daten in einen DataFrame (wie 'data') konvertieren.
df['class'], classes = pd.factorize(df['class'].str.decode('utf-8'))  # die 'class'-Spalte kategorisieren (Text -> Zahl)
data = data.reset_index(drop=True)                                    # die Nummerierung unserer 'data' neu vornehmen

df.equals(data) # prüft, ob der von dir aufbereitete Datensatz ´data´ mit der Musterlösung ´df´ übereinstimmt. Wenn ja, dann ist das Ergebnis "True"

Wenn dein Ergebnis "True" lautet, hast du die Datenvorbereitung erfolgreich abgeschlossen. Wir speichern nun die so vorbereiteten Daten als "Iris2.csv" Datei ab, um sie in der nächsten Übung verwenden zu können.

In [ ]:
data.to_csv('Iris2.csv', index=False)

Jetzt ist der Datensatz aufbereitet für die nächste praktische Übung zur Identifikation von Trainings- und Testdaten und dem Erstellen eines ML-Modells.